In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import sys
!pip install -Iv https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz#md5=61102459833cc31d6b05404325fa45a8

  Looking up "https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz" in the cache
  No cache entry available
  Starting new HTTPS connection (1): pypi.python.org
  "GET /packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz HTTP/1.1" 200 134993
    98% |############################### | 133kB 13.2MB/s eta 0:00:01  Updating cache with response from "https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz"
  Caching due to etag
    100% |################################| 143kB 2.6MB/s 
  Running setup.py (path:/tmp/pip-ucxguz41-build/setup.py) egg_info for package from https://pypi.python.org/packages/7b/98/7445165b69d4e95403372c07845ad1756af509ac85fc33c5b88e1c3f90c9/xlrd-0.9.0.tar.gz#md5=61102459833cc31d6b05404325fa45a8
    Running command python setup.py egg_info
    running egg_info
    creating pip-egg-info/xlrd.egg-info

In [4]:

# coding: utf-8

# In[1]:

import pandas as pd
import os 
import cv2
import numpy as np


# In[2]:

demographics = pd.read_excel('/src/Data/ANIML/oasis_cross-sectional.xls', sheetname=3) # load data
df = demographics.dropna(how='any') # remove NaN values
df_columns = list(demographics.columns)
X_columns = np.delete(df_columns, [6,7], None) # X matrix won't have MMSE or CDR scores
Xdf = df.reindex(columns=X_columns)
X = Xdf.values # creating X


# In[3]:

# Define function to get list of pngs based on slice number
pngs_path='/src/Data/ANIML/OASIS_pngs'
def getaxialPNG(path):
    l = []
    axialslice90_files = []
    axialslice91_files = []
    axialslice92_files = []
    axialslice93_files = []
    axialslice94_files = []
    axialslice95_files = []
    axialslice96_files = []
    axialslice97_files = []
    axialslice98_files = []
    axialslice99_files = []
    
    for root, directories, filenames in os.walk(path):
        
        for filename in filenames:
            if ".90." in filename: 
                axialslice90_files.append(os.path.join(root, filename))
            if ".91." in filename: 
                axialslice91_files.append(os.path.join(root, filename))
            if ".92." in filename: 
                axialslice92_files.append(os.path.join(root, filename))
            if ".93." in filename: 
                axialslice93_files.append(os.path.join(root, filename))
            if ".94." in filename: 
                axialslice94_files.append(os.path.join(root, filename))
            if ".95." in filename: 
                axialslice95_files.append(os.path.join(root, filename))
            if ".96." in filename: 
                axialslice96_files.append(os.path.join(root, filename))
            if ".97." in filename: 
                axialslice97_files.append(os.path.join(root, filename))
            if ".98." in filename: 
                axialslice98_files.append(os.path.join(root, filename))
            if ".99." in filename: 
                axialslice99_files.append(os.path.join(root, filename))

    l = list(zip(axialslice90_files, axialslice91_files, axialslice92_files, axialslice93_files, axialslice94_files, axialslice95_files, axialslice96_files, axialslice97_files, axialslice98_files, axialslice99_files))

    return ((np.asarray(l)))

axial_files0 = getaxialPNG(pngs_path)
axial_X_files = np.take(axial_files0, indices=df.index.values, axis=0) # keeps the images with the same index as X matrix
axial90loc, axial91loc, axial92loc, axial93loc, axial94loc, axial95loc, axial96loc, axial97loc, axial98loc, axial99loc = zip(*axial_X_files)


# In[4]:

X_id, X_sex, X_handedness, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF = zip(*X) # unzips big X matrix

def sex_translator(X_sex):
    X_sex_binary = []
    X_sex_encoded = []
    for x in X_sex:
        if x == 'M':
            X_sex_binary.append(1)
            X_sex_encoded.append([0,1])
        else:
            X_sex_binary.append(-1)
            X_sex_encoded.append([1,0])
    
    return(zip(X_sex_binary, X_sex_encoded)) # gives us binary and one-hot encoded for sex
           
def hand_translator(X_handedness):
    X_hand_binary = []
    X_hand_encoded = []
    for x in X_handedness:
        if x == 'R':
            X_hand_binary.append(1)
            X_hand_encoded.append([0,1])
        else:
            X_hand_binary.append(-1)
            X_hand_encoded.append([1,0])
    
    return(zip(X_hand_binary, X_hand_encoded)) # same as above but for handedness

# turns out all the patients are right-handed
# maybe we should all just be left-handed so we don't suffer from Alzheimer's
# /frequentist sarcasm

X_sex_binary, X_sex_encoded = zip(*sex_translator(X_sex)) # unzipping to get our function outputs
X_hand_binary, X_hand_encoded = zip(*hand_translator(X_handedness))


# In[5]:

def prepPNGimgs(array_of_image_paths):
    l = []
    for img_file in array_of_image_paths: # for each file in the list of images...
        img = cv2.imread("{}".format(img_file)) # read the image...
        img = cv2.resize(np.array(img), (224,224)) # resize it to 224 by 224 QUICK WHAT'S 224 SQUARED??
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # makes it grayscale
        flatten = gray.flatten() # spaghettifies it into 1 by 50176 (which is 224 squared)
        l.append(flatten)
    
    return(np.asarray(l))


# In[6]:

Y_CDR_columns = [column_name for column_name in df_columns if column_name == 'CDR']
Y_CDR_df = df.reindex(columns=Y_CDR_columns)
Y_CDR = Y_CDR_df.values

Y_MMSE_columns = [column_name for column_name in df_columns if column_name == 'MMSE']
Y_MMSE_df = df.reindex(columns=Y_MMSE_columns)
Y_MMSE = Y_MMSE_df.values

CDR_threshold_0 = 0 # threshold values by CDR scale
CDR_threshold_0point5 = 0.5
CDR_threshold_1 = 1

MMSE_threshold_24 = 24 # threshold values by MMSE scale
MMSE_threshold_18 = 18

def CDR_probable_AD_thresholder(Y_CDR, threshold_value):
    Y_CDR_binary = []
    Y_CDR_encoded = []
    for y in Y_CDR:
        if y > threshold_value:
            Y_CDR_binary.append(1)
            Y_CDR_encoded.append([0,1])
        else:
            Y_CDR_binary.append(-1)
            Y_CDR_encoded.append([1,0])

    return((zip(Y_CDR_binary, Y_CDR_encoded)))

def MMSE_probable_Dementia_thresholder(Y_MMSE, threshold_value):
    Y_MMSE_binary = []
    Y_MMSE_encoded = []
    for y in Y_MMSE:
        if y < threshold_value:
            Y_MMSE_binary.append(1)
            Y_MMSE_encoded.append([0,1])
        else:
            Y_MMSE_binary.append(-1)
            Y_MMSE_encoded.append([1,0])
        
    return(zip(Y_MMSE_binary, Y_MMSE_encoded))

Y_CDR_binary, Y_CDR_encoded = zip(*CDR_probable_AD_thresholder(Y_CDR, CDR_threshold_0))
Y_MMSE_binary, Y_MMSE_encoded = zip(*MMSE_probable_Dementia_thresholder(Y_MMSE, MMSE_threshold_24))


# In[7]:

# turning everything into numpy arrays because why not

df_index = np.asarray(df.index.values)
X_id = np.asarray(X_id)
X_sex = np.asarray(X_sex)
X_sex_binary = np.asarray(X_sex_binary)
X_sex_encoded = np.asarray(X_sex_encoded)
X_handedness = np.asarray(X_handedness)
X_hand_binary = np.asarray(X_hand_binary)
X_hand_encoded = np.asarray(X_hand_encoded)
X_age = np.asarray(X_age) 
X_education = np.asarray(X_education) 
X_SES = np.asarray(X_SES)
X_eTIV = np.asarray(X_eTIV)
X_nWBV = np.asarray(X_nWBV)
X_ASF = np.asarray(X_ASF)
axial90loc = np.asarray(axial90loc) 
axial91loc = np.asarray(axial91loc)
axial92loc = np.asarray(axial92loc) 
axial93loc = np.asarray(axial93loc)
axial94loc = np.asarray(axial94loc)
axial95loc = np.asarray(axial95loc) 
axial96loc = np.asarray(axial96loc) 
axial97loc = np.asarray(axial97loc) 
axial98loc = np.asarray(axial98loc) 
axial99loc = np.asarray(axial99loc)
axial90_spaghetti = prepPNGimgs(axial90loc)
axial91_spaghetti = prepPNGimgs(axial91loc)
axial92_spaghetti = prepPNGimgs(axial92loc)
axial93_spaghetti = prepPNGimgs(axial93loc)
axial94_spaghetti = prepPNGimgs(axial94loc)
axial95_spaghetti = prepPNGimgs(axial95loc)
axial96_spaghetti = prepPNGimgs(axial96loc)
axial97_spaghetti = prepPNGimgs(axial97loc)
axial98_spaghetti = prepPNGimgs(axial98loc)
axial99_spaghetti = prepPNGimgs(axial99loc)
Y_CDR = np.squeeze(np.asarray(Y_CDR))
Y_CDR_binary = np.asarray(Y_CDR_binary)
Y_CDR_encoded = np.asarray(Y_CDR_encoded)
Y_MMSE = np.squeeze(np.asarray(Y_MMSE))
Y_MMSE_binary = np.asarray(Y_MMSE_binary)
Y_MMSE_encoded = np.asarray(Y_MMSE_encoded)


# In[9]:

FinalX = np.vstack((X_id, X_sex, X_sex_binary, X_handedness, X_hand_binary, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF))
FinalX = FinalX.T

FinalY = np.vstack((Y_CDR, Y_CDR_binary, Y_MMSE, Y_MMSE_binary))
FinalY = FinalY.T

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
XDATAFRAME = pd.DataFrame((FinalX), index=df_index, columns=['X_id', 'X_sex', 'X_sex_binary', 'X_handedness', 'X_hand_binary', 'X_age', 'X_education', 'X_SES', 'X_eTIV', 'X_nWBV', 'X_ASF'])

YDATAFRAME = pd.DataFrame((FinalY), index=df_index, columns=['Y_CDR', 'Y_CDR_binary', 'Y_MMSE', 'Y_MMSE_binary'])

classifiers = pd.DataFrame((Y_CDR_encoded), index=df_index, columns=['NC','AD'])


In [6]:
axial90s = pd.concat([pd.DataFrame((axial90_spaghetti), index=df_index), classifiers], axis=1)
axial91s = pd.concat([pd.DataFrame((axial91_spaghetti), index=df_index), classifiers], axis=1)
axial92s = pd.concat([pd.DataFrame((axial92_spaghetti), index=df_index), classifiers], axis=1)
axial93s = pd.concat([pd.DataFrame((axial93_spaghetti), index=df_index), classifiers], axis=1)
axial94s = pd.concat([pd.DataFrame((axial94_spaghetti), index=df_index), classifiers], axis=1)
axial95s = pd.concat([pd.DataFrame((axial95_spaghetti), index=df_index), classifiers], axis=1)
axial96s = pd.concat([pd.DataFrame((axial96_spaghetti), index=df_index), classifiers], axis=1)
axial97s = pd.concat([pd.DataFrame((axial97_spaghetti), index=df_index), classifiers], axis=1)
axial98s = pd.concat([pd.DataFrame((axial98_spaghetti), index=df_index), classifiers], axis=1)
axial99s = pd.concat([pd.DataFrame((axial99_spaghetti), index=df_index), classifiers], axis=1)

# axial91s = pd.DataFrame((axial91_spaghetti), index=df_index)
# axial92s = pd.DataFrame((axial92_spaghetti), index=df_index)
# axial93s = pd.DataFrame((axial93_spaghetti), index=df_index)
# axial94s = pd.DataFrame((axial94_spaghetti), index=df_index)
# axial95s = pd.DataFrame((axial95_spaghetti), index=df_index)
# axial96s = pd.DataFrame((axial96_spaghetti), index=df_index)
# axial97s = pd.DataFrame((axial97_spaghetti), index=df_index)
# axial98s = pd.DataFrame((axial98_spaghetti), index=df_index)
# axial99s = pd.DataFrame((axial99_spaghetti), index=df_index)

# divide subject groups into training and test


# allSlices = pd.concat([axial90s, axial91s, axial93s, axial94s, axial95s, axial96s, axial97s, axial98s, axial99s],axis=0)

# #shuffle rows
# allSlicesShuff = allSlices.sample(frac=1, random_state=1).reset_index(drop=True)

# X_train0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[0:1000].as_matrix()
# y_train0 = allSlicesShuff['AD'][0:1000].as_matrix()

# X_test0 = allSlicesShuff.drop(['NC', 'AD'], axis=1)[1000:].as_matrix()
# y_test0 = allSlicesShuff['AD'][1000:].as_matrix()

In [7]:
import random
# axial90s.shape
ind= range(0,216)
random.seed(1)
train_ind = random.sample(range(0,216),200)

axials_train = {}
axials_train[1] = axial90s.take(train_ind)
axials_train[2] = axial91s.take(train_ind)
axials_train[3] = axial92s.take(train_ind)
axials_train[4] = axial93s.take(train_ind)
axials_train[5] = axial94s.take(train_ind)
axials_train[6] = axial95s.take(train_ind)
axials_train[7] = axial96s.take(train_ind)
axials_train[8] = axial97s.take(train_ind)
axials_train[9] = axial98s.take(train_ind)
axials_train[10] = axial99s.take(train_ind)


test_ind = list(set(ind) - set(train_ind))

axials_test={}
axials_test[1] = axial90s.take(test_ind)
axials_test[2] = axial91s.take(test_ind)
axials_test[3] = axial92s.take(test_ind)
axials_test[4] = axial93s.take(test_ind)
axials_test[5] = axial94s.take(test_ind)
axials_test[6] = axial95s.take(test_ind)
axials_test[7] = axial96s.take(test_ind)
axials_test[8] = axial97s.take(test_ind)
axials_test[9] = axial98s.take(test_ind)
axials_test[10] = axial99s.take(test_ind)


In [8]:
trainDF = pd.concat(axials_train)
testDF = pd.concat(axials_test)

X_train0 = trainDF.drop(['NC', 'AD'], axis=1).as_matrix()
y_train0 = trainDF['AD'].as_matrix()

X_test0 = testDF.drop(['NC', 'AD'], axis=1).as_matrix()
y_test0 = testDF['AD'].as_matrix()

In [9]:
X_train, y_train, X_test, y_test = X_train0, y_train0, X_test0, y_test0
X_train.shape

(2000, 50176)

In [12]:
class LeNetMnistClassifier():
    @staticmethod
    def toDigit(hot_encode):
        return np.argmax(hot_encode)
        
    def __init__(self, **kwargs):
        if kwargs.get("model_path", None):
            from keras.models import load_model
            self.model = load_model(kwargs['model_path'])
        else:
            self.activation_function = kwargs.pop('activation_function', 'relu')
            self.batch_size = kwargs.pop('batch_size', 30)
            self.epochs = kwargs.pop('epochs', 5)
            self.kernal_size = kwargs.pop('kernal_size', (3, 3))
            self.loss_function = kwargs.pop('loss_function', 'categorical_crossentropy')
            self.optimizer = kwargs.pop('optimizer', 'sgd')
            self.pool_size = kwargs.pop('pool_size', (2, 2))
            self.model = self._model()
            
    
    def _model(self):
        model = Sequential()
        model.add(Conv2D(32, self.kernal_size[0],self.kernal_size[1] , border_mode='same',
                 activation=self.activation_function,
                 input_shape=(224,224,1)))
        # One additional convolutional layer (32 channels)
        model.add(Conv2D(32, self.kernal_size[0],self.kernal_size[1], border_mode='same',
                 activation=self.activation_function))
        model.add(Conv2D(32, self.kernal_size[0],self.kernal_size[1], border_mode='same',
                 activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Conv2D(64, self.kernal_size[0],self.kernal_size[1], border_mode='same', activation=self.activation_function))
        # One additional convolutional layer (64 channels)
        model.add(Conv2D(64, self.kernal_size[0],self.kernal_size[1], border_mode='same', activation=self.activation_function))
        model.add(Conv2D(64, self.kernal_size[0],self.kernal_size[1], border_mode='same', activation=self.activation_function))
        model.add(MaxPooling2D(pool_size=self.pool_size))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(256, activation=self.activation_function))
        model.add(Dropout(0.25))
        model.add(Dense(2, activation='softmax'))
        
        return model
    
    def predict(self, digits=()):
        return self.model.predict(digits)
    
    def evaluate(self, X_test=None, y_test=None):
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)
        X_test = X_test.astype('float32')
        X_test/=255        
        number_of_classes = 2
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        return self.model.evaluate(X_test, y_test)
    
    def preprocess_and_train(self, X_train=None, y_train=None, X_test=None, y_test=None):
        self._train(*self._preprocess(X_train, y_train, X_test, y_test))
        
    def _preprocess(self, X_train, y_train, X_test, y_test):
        X_train = X_train.reshape(X_train.shape[0], 224, 224, 1)
        X_test = X_test.reshape(X_test.shape[0], 224, 224, 1)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        
        X_train/=255
        X_test/=255
        
        number_of_classes = 2
        y_train = np_utils.to_categorical(y_train, number_of_classes)
        y_test = np_utils.to_categorical(y_test, number_of_classes)
        
        return X_train, y_train, X_test, y_test
    
    def _train(self, X_train, y_train, X_test, y_test):
        self.model.compile(loss=self.loss_function,
              optimizer=self.optimizer,
              metrics=['accuracy'])
        
        fit_output = self.model.fit(X_train,
                        y_train,
                        batch_size=self.batch_size,
                        nb_epoch=self.epochs,
                        verbose=1,
                        validation_data=(X_test, y_test))
        self._history = fit_output.history
        
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        self.model.save("oasis_test_{0}.h5".format(timestr)) 
    
    @property
    def history(self):
        return self._history

In [13]:
classifier = LeNetMnistClassifier(epochs=30, optimizer=keras.optimizers.Adadelta())
classifier.preprocess_and_train(X_train=X_train0, y_train=y_train0, X_test=X_test0, y_test=y_test0)

Train on 2000 samples, validate on 160 samples
Epoch 1/30
2000/2000 [==============================] - 16s - loss: 0.6794 - acc: 0.6080 - val_loss: 0.6699 - val_acc: 0.6250
Epoch 2/30
2000/2000 [==============================] - 14s - loss: 0.6715 - acc: 0.6150 - val_loss: 0.6602 - val_acc: 0.6250
Epoch 3/30
2000/2000 [==============================] - 14s - loss: 0.6711 - acc: 0.6150 - val_loss: 0.6652 - val_acc: 0.6250
Epoch 4/30
2000/2000 [==============================] - 14s - loss: 0.6705 - acc: 0.6150 - val_loss: 0.6589 - val_acc: 0.6250
Epoch 5/30
2000/2000 [==============================] - 14s - loss: 0.6654 - acc: 0.6150 - val_loss: 0.6621 - val_acc: 0.6250
Epoch 6/30
2000/2000 [==============================] - 14s - loss: 0.6551 - acc: 0.6125 - val_loss: 0.6501 - val_acc: 0.6250
Epoch 7/30
2000/2000 [==============================] - 14s - loss: 0.6195 - acc: 0.6620 - val_loss: 0.6503 - val_acc: 0.6000
Epoch 8/30
2000/2000 [==============================] - 14s - loss: 0.5

In [2]:
import tensorflow as tf
tf.Session(config=tf.ConfigProto(log_device_placement=True))

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

print(tf.__version__)

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13154147634325830913
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 104136704
locality {
  bus_id: 1
}
incarnation: 18100578659675994095
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0"
]
1.2.1


In [ ]:
## prepare data set
